# ✅ Validación de Datos con LLMs

Objetivo: usar LLMs para detectar anomalías, validar calidad de datos, clasificar errores, y generar reglas de validación de forma inteligente.

- Duración: 90 min
- Dificultad: Media/Alta
- Stack: OpenAI, Great Expectations, Pandas

## 🔍 LLMs para Data Quality: Más Allá de Reglas Estáticas

La **validación tradicional** de datos se basa en reglas predefinidas (nulls, rangos, formatos). Los **LLMs** permiten validación **semántica y contextual**: detectar anomalías que ninguna regla puede capturar, explicar root causes, y generar reglas de validación automáticamente.

### 🏗️ Evolución de Data Quality

```
┌─────────────────────────────────────────────────────────────────┐
│         DATA QUALITY: TRADITIONAL vs LLM-POWERED                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  TRADITIONAL (Rule-Based):                                       │
│  ┌──────────────────────────────────────────────────────────┐   │
│  │ 1. Null Checks                                           │   │
│  │    assert column.notnull().all()                         │   │
│  │    ✅ Detecta: NULL values                               │   │
│  │    ❌ No detecta: "N/A", "Unknown", "—"                 │   │
│  │                                                          │   │
│  │ 2. Range Checks                                          │   │
│  │    assert (edad >= 0) & (edad <= 120)                    │   │
│  │    ✅ Detecta: edad = -5 o 200                           │   │
│  │    ❌ No detecta: edad = 999 (typo de 99)               │   │
│  │                                                          │   │
│  │ 3. Regex Validation                                      │   │
│  │    assert email.str.match(r'^[\w.-]+@[\w.-]+\.\w+$')    │   │
│  │    ✅ Detecta: "invalid-email"                           │   │
│  │    ❌ No detecta: "test@test.test" (fake)               │   │
│  │                                                          │   │
│  │ 4. Referential Integrity                                 │   │
│  │    assert ventas.producto_id.isin(productos.id)          │   │
│  │    ✅ Detecta: producto_id = 999 (no existe)            │   │
│  │    ❌ No explica POR QUÉ falló                          │   │
│  └──────────────────────────────────────────────────────────┘   │
│                                                                  │
│  LLM-POWERED (Semantic + Contextual):                            │
│  ┌──────────────────────────────────────────────────────────┐   │
│  │ 1. Semantic Null Detection                               │   │
│  │    LLM analiza: "N/A", "Unknown", "—", "TBD"            │   │
│  │    → Detecta pseudo-nulls con contexto                   │   │
│  │    ✅ "N/A" en columna 'ciudad' → NULL semántico        │   │
│  │    ✅ "N/A" en columna 'notas' → Válido                 │   │
│  │                                                          │   │
│  │ 2. Contextual Outlier Detection                          │   │
│  │    LLM: "edad = 999 es typo de 99 (patrón común)"       │   │
│  │    → Sugiere corrección basada en contexto              │   │
│  │    ✅ Explica: "Probablemente typo en entrada manual"   │   │
│  │    ✅ Sugiere: 99 o 9 (según distribución)              │   │
│  │                                                          │   │
│  │ 3. Plausibility Validation                               │   │
│  │    LLM: "test@test.test es email técnico, no real"      │   │
│  │    → Detecta datos sintéticamente válidos pero falsos   │   │
│  │    ✅ "admin@example.com" → Test data                   │   │
│  │    ✅ "Nueva Yorkk" → Typo de "Nueva York"              │   │
│  │                                                          │   │
│  │ 4. Root Cause Analysis                                   │   │
│  │    LLM: "50 producto_ids huérfanos desde 2024-10-15"    │   │
│  │    → Analiza: "Coincide con deploy de API v2"           │   │
│  │    ✅ Explica causa probable                            │   │
│  │    ✅ Sugiere solución: "Mapear IDs antiguos→nuevos"    │   │
│  └──────────────────────────────────────────────────────────┘   │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### 🎯 Casos de Uso: LLMs vs Traditional

| Problema | Traditional | LLM-Powered | Ganador |
|----------|-------------|-------------|---------|
| **NULL detection** | `column.isnull()` | Detecta "N/A", "Unknown", "—" | 🏆 LLM |
| **Email validation** | Regex pattern | Detecta test@test.com como fake | 🏆 LLM |
| **City names** | Whitelist de ciudades | Detecta "Nueva Yorkk" como typo | 🏆 LLM |
| **Age range** | `age.between(0, 120)` | "999 es typo de 99" | 🏆 LLM |
| **Performance** | Instantáneo | 100-500ms por validación | 🏆 Traditional |
| **Costo** | Gratis | $0.001-$0.01 por registro | 🏆 Traditional |
| **Explicabilidad** | Regla booleana | Natural language explanation | 🏆 LLM |

**Conclusión**: LLMs **complementan** (no reemplazan) validación tradicional. Usar ambos en **híbrido**.

### 🔧 Implementación: Sistema Híbrido de Validación

```python
from typing import List, Dict, Optional, Literal
from pydantic import BaseModel, Field
from openai import OpenAI
import pandas as pd
import numpy as np
from dataclasses import dataclass
from enum import Enum
import json

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# 1. MODELOS DE DATOS

class ValidationSeverity(str, Enum):
    """Severidad de issue de calidad."""
    CRITICAL = "CRITICAL"  # Bloquea pipeline
    ERROR = "ERROR"        # Requiere fix
    WARNING = "WARNING"    # Revisar
    INFO = "INFO"          # Informativo

class ValidationIssue(BaseModel):
    """Issue de calidad detectado."""
    column: str
    row_index: Optional[int] = None
    value: Optional[str] = None
    issue_type: str  # NULLS, OUTLIER, FORMAT, INCONSISTENCY, etc.
    severity: ValidationSeverity
    confidence: float = Field(..., ge=0, le=1)
    description: str
    suggested_fix: Optional[str] = None
    detected_by: Literal["traditional", "llm", "hybrid"]

class ValidationReport(BaseModel):
    """Reporte completo de validación."""
    table_name: str
    total_rows: int
    total_issues: int
    issues_by_severity: Dict[ValidationSeverity, int]
    issues: List[ValidationIssue]
    execution_time_seconds: float
    cost_usd: float

# 2. VALIDADORES TRADICIONALES

class TraditionalValidator:
    """Validador basado en reglas estáticas."""
    
    @staticmethod
    def check_nulls(df: pd.DataFrame, column: str) -> List[ValidationIssue]:
        """Detecta valores NULL."""
        null_mask = df[column].isnull()
        null_count = null_mask.sum()
        
        if null_count == 0:
            return []
        
        # Detectar todos los nulls
        issues = []
        for idx in df[null_mask].index[:100]:  # Limitar a 100 por performance
            issues.append(ValidationIssue(
                column=column,
                row_index=int(idx),
                value=None,
                issue_type="NULLS",
                severity=ValidationSeverity.ERROR,
                confidence=1.0,
                description=f"Valor NULL en columna requerida '{column}'",
                suggested_fix="Impute con media/moda o remover registro",
                detected_by="traditional"
            ))
        
        return issues
    
    @staticmethod
    def check_range(
        df: pd.DataFrame, 
        column: str, 
        min_val: float, 
        max_val: float
    ) -> List[ValidationIssue]:
        """Detecta valores fuera de rango."""
        out_of_range = df[(df[column] < min_val) | (df[column] > max_val)]
        
        issues = []
        for idx, row in out_of_range.iterrows():
            value = row[column]
            issues.append(ValidationIssue(
                column=column,
                row_index=int(idx),
                value=str(value),
                issue_type="OUTLIER",
                severity=ValidationSeverity.ERROR,
                confidence=1.0,
                description=f"Valor {value} fuera de rango [{min_val}, {max_val}]",
                suggested_fix=f"Clamp a rango o marcar como outlier",
                detected_by="traditional"
            ))
        
        return issues
    
    @staticmethod
    def check_regex(
        df: pd.DataFrame, 
        column: str, 
        pattern: str
    ) -> List[ValidationIssue]:
        """Valida formato con regex."""
        invalid_mask = ~df[column].astype(str).str.match(pattern)
        invalid_values = df[invalid_mask]
        
        issues = []
        for idx, row in invalid_values.iterrows():
            value = row[column]
            issues.append(ValidationIssue(
                column=column,
                row_index=int(idx),
                value=str(value),
                issue_type="FORMAT",
                severity=ValidationSeverity.ERROR,
                confidence=1.0,
                description=f"Valor '{value}' no coincide con patrón esperado",
                suggested_fix="Reformatear según pattern",
                detected_by="traditional"
            ))
        
        return issues

# 3. VALIDADOR CON LLM

class LLMValidator:
    """Validador con análisis semántico usando LLM."""
    
    def __init__(self, model: str = "gpt-4o-mini"):  # Modelo barato para validación
        self.model = model
        self.total_cost = 0.0
    
    def check_semantic_nulls(
        self, 
        df: pd.DataFrame, 
        column: str
    ) -> List[ValidationIssue]:
        """
        Detecta valores que son NULL semánticamente (N/A, Unknown, TBD, etc.)
        pero no NULL sintácticamente.
        """
        # Obtener valores únicos no-null
        non_null_values = df[column].dropna().unique()[:50]  # Limitar muestra
        
        prompt = f"""Analiza estos valores de la columna '{column}' y detecta cuáles son NULL semánticos (representan faltante/desconocido/no aplicable):

Valores: {list(non_null_values)}

Responde en JSON:
{{
  "semantic_nulls": ["valor1", "valor2", ...],
  "confidence": 0.0-1.0,
  "reasoning": "explicación breve"
}}"""
        
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            
            # Calcular costo aproximado (gpt-4o-mini: $0.15/$0.60 por 1M tokens)
            input_tokens = len(prompt) / 4  # Aproximación
            output_tokens = len(response.choices[0].message.content) / 4
            cost = (input_tokens / 1_000_000 * 0.15) + (output_tokens / 1_000_000 * 0.60)
            self.total_cost += cost
            
            # Crear issues para cada NULL semántico
            issues = []
            for null_value in result.get("semantic_nulls", []):
                mask = df[column] == null_value
                for idx in df[mask].index[:100]:
                    issues.append(ValidationIssue(
                        column=column,
                        row_index=int(idx),
                        value=str(null_value),
                        issue_type="SEMANTIC_NULL",
                        severity=ValidationSeverity.WARNING,
                        confidence=result.get("confidence", 0.8),
                        description=f"'{null_value}' es NULL semántico: {result.get('reasoning', '')}",
                        suggested_fix="Convertir a NULL explícito",
                        detected_by="llm"
                    ))
            
            return issues
            
        except Exception as e:
            print(f"Error en LLM validation: {e}")
            return []
    
    def check_plausibility(
        self, 
        df: pd.DataFrame, 
        column: str, 
        context: str = ""
    ) -> List[ValidationIssue]:
        """
        Valida plausibilidad de valores (ej. emails fake, ciudades con typos).
        """
        sample_values = df[column].dropna().head(20).tolist()
        
        prompt = f"""Analiza la plausibilidad de estos valores en la columna '{column}':

Contexto: {context}
Valores: {sample_values}

Detecta valores implausibles (fake, typos, test data, etc.).

Responde en JSON:
{{
  "implausible_values": [
    {{
      "value": "valor",
      "reason": "explicación",
      "confidence": 0.0-1.0,
      "suggested_fix": "corrección o null"
    }}
  ]
}}"""
        
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            
            # Costo
            cost = (len(prompt) / 4 / 1_000_000 * 0.15) + (len(response.choices[0].message.content) / 4 / 1_000_000 * 0.60)
            self.total_cost += cost
            
            issues = []
            for item in result.get("implausible_values", []):
                mask = df[column] == item["value"]
                for idx in df[mask].index:
                    issues.append(ValidationIssue(
                        column=column,
                        row_index=int(idx),
                        value=str(item["value"]),
                        issue_type="IMPLAUSIBLE",
                        severity=ValidationSeverity.WARNING,
                        confidence=item.get("confidence", 0.7),
                        description=f"Valor implausible: {item.get('reason', '')}",
                        suggested_fix=item.get("suggested_fix"),
                        detected_by="llm"
                    ))
            
            return issues
            
        except Exception as e:
            print(f"Error en plausibility check: {e}")
            return []
    
    def explain_outlier(
        self, 
        value: float, 
        column: str,
        stats: Dict[str, float]
    ) -> str:
        """Explica por qué un valor es outlier en lenguaje natural."""
        prompt = f"""Explica por qué este valor es anómalo en 2-3 frases:

Columna: {column}
Valor: {value}

Estadísticas:
- Media: {stats['mean']:.2f}
- Desv. estándar: {stats['std']:.2f}
- Min: {stats['min']:.2f}
- Max (sin este): {stats['max']:.2f}
- Percentil 99: {stats.get('p99', 'N/A')}

Incluye posible causa raíz (typo, error de medición, evento real extremo, etc.)."""
        
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3
            )
            
            cost = (len(prompt) / 4 / 1_000_000 * 0.15) + (len(response.choices[0].message.content) / 4 / 1_000_000 * 0.60)
            self.total_cost += cost
            
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            return f"Error generando explicación: {e}"

# 4. VALIDADOR HÍBRIDO (COMBINA AMBOS)

class HybridValidator:
    """Sistema híbrido: Traditional (rápido) + LLM (semántico)."""
    
    def __init__(self):
        self.traditional = TraditionalValidator()
        self.llm = LLMValidator()
    
    def validate_dataframe(
        self, 
        df: pd.DataFrame, 
        table_name: str,
        validation_config: Dict
    ) -> ValidationReport:
        """
        Valida DataFrame completo con estrategia híbrida.
        
        Args:
            df: DataFrame a validar
            table_name: nombre de la tabla
            validation_config: configuración por columna
                {
                    "column_name": {
                        "checks": ["nulls", "range", "semantic_nulls", "plausibility"],
                        "range": [0, 120],
                        "context": "descripción para LLM"
                    }
                }
        """
        import time
        start_time = time.time()
        
        all_issues = []
        
        for column, config in validation_config.items():
            if column not in df.columns:
                continue
            
            checks = config.get("checks", [])
            
            # TRADITIONAL CHECKS (siempre primero - rápidos)
            if "nulls" in checks:
                all_issues.extend(self.traditional.check_nulls(df, column))
            
            if "range" in checks and "range" in config:
                min_val, max_val = config["range"]
                all_issues.extend(self.traditional.check_range(df, column, min_val, max_val))
            
            if "regex" in checks and "pattern" in config:
                all_issues.extend(self.traditional.check_regex(df, column, config["pattern"]))
            
            # LLM CHECKS (solo si hay issues o configurado explícitamente)
            if "semantic_nulls" in checks:
                all_issues.extend(self.llm.check_semantic_nulls(df, column))
            
            if "plausibility" in checks:
                context = config.get("context", f"Columna {column}")
                all_issues.extend(self.llm.check_plausibility(df, column, context))
        
        # Generar reporte
        execution_time = time.time() - start_time
        
        issues_by_severity = {
            severity: sum(1 for issue in all_issues if issue.severity == severity)
            for severity in ValidationSeverity
        }
        
        report = ValidationReport(
            table_name=table_name,
            total_rows=len(df),
            total_issues=len(all_issues),
            issues_by_severity=issues_by_severity,
            issues=all_issues,
            execution_time_seconds=execution_time,
            cost_usd=self.llm.total_cost
        )
        
        return report

# EJEMPLO DE USO

# Dataset de prueba con varios tipos de problemas
df_test = pd.DataFrame({
    'customer_id': [1, 2, 3, 4, 5, 6, 7, 8],
    'age': [25, 30, 999, 45, None, 22, -5, 150],  # Outliers, null, negativos
    'email': [
        'user@example.com',
        'admin@test.test',  # Fake
        'invalid-email',     # Inválido
        'test@test.com',     # Test data
        None,
        'real@company.io',
        'N/A',               # NULL semántico
        'john@gmail.com'
    ],
    'city': [
        'New York',
        'Nueva Yorkk',  # Typo
        'Los Angeles',
        'Unknown',      # NULL semántico
        'San Francisco',
        'N/A',          # NULL semántico
        'Chicago',
        'TBD'           # NULL semántico
    ]
})

# Configuración de validación
validation_config = {
    'age': {
        'checks': ['nulls', 'range'],
        'range': [0, 120]
    },
    'email': {
        'checks': ['nulls', 'regex', 'plausibility'],
        'pattern': r'^[\w\.-]+@[\w\.-]+\.\w+$',
        'context': 'Emails de clientes reales (no test data)'
    },
    'city': {
        'checks': ['semantic_nulls', 'plausibility'],
        'context': 'Ciudades de USA'
    }
}

# Ejecutar validación híbrida
validator = HybridValidator()
report = validator.validate_dataframe(df_test, 'customers', validation_config)

# Mostrar resultados
print(f"📊 REPORTE DE VALIDACIÓN: {report.table_name}")
print(f"   Total filas: {report.total_rows}")
print(f"   Total issues: {report.total_issues}")
print(f"   Tiempo: {report.execution_time_seconds:.2f}s")
print(f"   Costo LLM: ${report.cost_usd:.4f}")
print(f"\n   Issues por severidad:")
for severity, count in report.issues_by_severity.items():
    if count > 0:
        print(f"     {severity.value}: {count}")

print(f"\n🔍 ISSUES DETECTADOS (top 10):")
for i, issue in enumerate(report.issues[:10], 1):
    print(f"\n{i}. [{issue.severity.value}] {issue.issue_type}")
    print(f"   Columna: {issue.column}, Fila: {issue.row_index}")
    print(f"   Valor: {issue.value}")
    print(f"   {issue.description}")
    print(f"   Confianza: {issue.confidence:.0%} | Detectado por: {issue.detected_by}")
    if issue.suggested_fix:
        print(f"   💡 Fix sugerido: {issue.suggested_fix}")
```

### 📊 Comparación de Performance

```python
import time

# Benchmark: Traditional vs LLM
df_large = pd.DataFrame({
    'age': np.random.randint(18, 70, size=10000),
    'email': ['user{}@example.com'.format(i) for i in range(10000)]
})

# Traditional validation
start = time.time()
trad_validator = TraditionalValidator()
trad_issues = trad_validator.check_range(df_large, 'age', 0, 120)
trad_time = time.time() - start

print(f"Traditional: {trad_time*1000:.2f}ms, {len(trad_issues)} issues, $0")

# LLM validation (solo muestra)
start = time.time()
llm_validator = LLMValidator()
llm_issues = llm_validator.check_plausibility(df_large.head(20), 'email', 'Customer emails')
llm_time = time.time() - start

print(f"LLM: {llm_time*1000:.2f}ms, {len(llm_issues)} issues, ${llm_validator.total_cost:.4f}")
print(f"\n💡 LLM es {llm_time/trad_time:.0f}x más lento pero detecta issues semánticos")
```

### 🎯 Estrategia de Uso Óptima

```python
# REGLA DE ORO: Híbrido inteligente

# 1. TRADITIONAL FIRST (always)
#    - Rápido, gratis, confiable
#    - Cubre 80% de casos

# 2. LLM IF:
#    a) Traditional encontró issues ambiguos
#    b) Columnas críticas (PII, identificadores, nombres)
#    c) Sample pequeño (<1000 registros únicos)
#    d) Budget disponible ($0.01-$0.10 por tabla)

# Ejemplo de decisión:
def should_use_llm(df: pd.DataFrame, column: str, trad_issues: int) -> bool:
    """Decide si vale la pena usar LLM."""
    unique_count = df[column].nunique()
    
    # Criterios para usar LLM:
    if trad_issues > 0 and trad_issues < 100:  # Issues moderados
        return True
    
    if unique_count < 100:  # Pocos valores únicos (categóricos)
        return True
    
    if column in ['email', 'name', 'city', 'address']:  # Campos críticos
        return True
    
    return False
```

### 🚀 Mejores Prácticas

1. **Siempre traditional primero**: Fast fail para issues obvios
2. **LLM como segunda capa**: Solo para casos ambiguos o semánticos
3. **Cache agresivo**: Mismos valores → misma respuesta (lru_cache)
4. **Batch validation**: Validar múltiples valores en un solo prompt
5. **Confidence thresholds**: Solo actuar si confidence >0.8
6. **Human review**: Issues con confidence <0.9 requieren revisión
7. **Cost monitoring**: Alertar si costo >$1 por tabla
8. **A/B testing**: Comparar LLM vs tradicional en métricas de negocio

## 🤖 Generación Automática de Reglas de Validación con LLMs

Escribir reglas de validación manualmente es tedioso y propenso a errores. Los **LLMs** pueden **inferir reglas de validación** analizando datos y generando código de Great Expectations, dbt tests, o Pandas assertions automáticamente.

### 🏗️ Arquitectura de Auto-Generación de Reglas

```
┌─────────────────────────────────────────────────────────────────┐
│      AUTO-GENERATION: FROM DATA TO VALIDATION RULES              │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  INPUT: DataFrame con datos históricos                          │
│  ┌──────────────────────────────────────────────────────────┐   │
│  │ df['age']: [25, 30, 45, 22, 28, ...]                     │   │
│  │ df['email']: ['user@x.com', 'admin@y.org', ...]          │   │
│  │ df['status']: ['active', 'active', 'inactive', ...]      │   │
│  └──────────────────────────────────────────────────────────┘   │
│        ↓                                                         │
│  1️⃣ PROFILE DATA (EDA automático)                              │
│     ┌────────────────────────────────────────────────────┐      │
│     │ pandas-profiling / ydata-profiling                 │      │
│     │ • Tipos de datos (int, str, datetime)             │      │
│     │ • Distribuciones (mean, std, quantiles)           │      │
│     │ • Valores únicos y frecuencias                    │      │
│     │ • Nulls, duplicates, outliers                     │      │
│     │ • Correlaciones entre columnas                    │      │
│     └────────────────────────────────────────────────────┘      │
│        ↓                                                         │
│  2️⃣ INFER RULES (LLM analiza profile)                          │
│     ┌────────────────────────────────────────────────────┐      │
│     │ Prompt: "Basándote en estas estadísticas,         │      │
│     │          genera reglas de validación..."           │      │
│     │                                                    │      │
│     │ LLM → Reglas inferidas:                           │      │
│     │  age:                                              │      │
│     │    - No NULL (0% nulls observed)                  │      │
│     │    - Range [18, 65] (min=18, max=65, no outliers)│      │
│     │    - Integer type                                 │      │
│     │                                                    │      │
│     │  email:                                            │      │
│     │    - Match regex ^[\w.-]+@[\w.-]+\.\w+$          │      │
│     │    - No duplicates (100% unique)                  │      │
│     │    - Max length 100 chars                         │      │
│     │                                                    │      │
│     │  status:                                           │      │
│     │    - IN ['active', 'inactive', 'pending']         │      │
│     │    - No NULL (0% nulls)                           │      │
│     └────────────────────────────────────────────────────┘      │
│        ↓                                                         │
│  3️⃣ GENERATE CODE (Target framework)                           │
│     ┌────────────────────────────────────────────────────┐      │
│     │ Great Expectations:                                │      │
│     │ ```python                                          │      │
│     │ suite.expect_column_values_to_not_be_null('age')  │      │
│     │ suite.expect_column_values_to_be_between(         │      │
│     │     'age', min_value=18, max_value=65)            │      │
│     │ suite.expect_column_values_to_match_regex(        │      │
│     │     'email', regex='^[\w.-]+@[\w.-]+\.\w+$')     │      │
│     │ ```                                                │      │
│     │                                                    │      │
│     │ dbt tests:                                         │      │
│     │ ```yaml                                            │      │
│     │ - name: age                                        │      │
│     │   tests:                                           │      │
│     │     - not_null                                     │      │
│     │     - dbt_utils.accepted_range:                   │      │
│     │         min_value: 18                              │      │
│     │         max_value: 65                              │      │
│     │ ```                                                │      │
│     └────────────────────────────────────────────────────┘      │
│        ↓                                                         │
│  4️⃣ VALIDATE & REFINE                                          │
│     - Ejecutar reglas contra datos nuevos                       │
│     - Si >5% false positives → Ajustar thresholds               │
│     - Human review de reglas generadas                          │
│        ↓                                                         │
│  OUTPUT: Validation suite lista para producción                 │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### 🔧 Implementación Completa

```python
from typing import Dict, List, Optional
from pydantic import BaseModel
import pandas as pd
import numpy as np
from openai import OpenAI
import json

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# 1. DATA PROFILER

class ColumnProfile(BaseModel):
    """Profile estadístico de una columna."""
    name: str
    dtype: str
    count: int
    null_count: int
    null_percentage: float
    unique_count: int
    unique_percentage: float
    
    # Numérico
    mean: Optional[float] = None
    std: Optional[float] = None
    min: Optional[float] = None
    max: Optional[float] = None
    q25: Optional[float] = None
    q50: Optional[float] = None
    q75: Optional[float] = None
    
    # String
    avg_length: Optional[float] = None
    max_length: Optional[int] = None
    
    # Categorical
    top_values: Optional[Dict[str, int]] = None
    
    # Ejemplos
    sample_values: List[str] = []

class DataProfiler:
    """Genera profile estadístico de DataFrame."""
    
    @staticmethod
    def profile_column(df: pd.DataFrame, column: str) -> ColumnProfile:
        """Genera profile de una columna."""
        col_data = df[column]
        
        profile = ColumnProfile(
            name=column,
            dtype=str(col_data.dtype),
            count=len(col_data),
            null_count=int(col_data.isnull().sum()),
            null_percentage=float(col_data.isnull().mean()),
            unique_count=int(col_data.nunique()),
            unique_percentage=float(col_data.nunique() / len(col_data)),
            sample_values=[str(v) for v in col_data.dropna().head(5).tolist()]
        )
        
        # Stats numéricos
        if pd.api.types.is_numeric_dtype(col_data):
            profile.mean = float(col_data.mean())
            profile.std = float(col_data.std())
            profile.min = float(col_data.min())
            profile.max = float(col_data.max())
            profile.q25 = float(col_data.quantile(0.25))
            profile.q50 = float(col_data.quantile(0.50))
            profile.q75 = float(col_data.quantile(0.75))
        
        # Stats string
        if pd.api.types.is_string_dtype(col_data) or col_data.dtype == 'object':
            str_lengths = col_data.dropna().astype(str).str.len()
            if len(str_lengths) > 0:
                profile.avg_length = float(str_lengths.mean())
                profile.max_length = int(str_lengths.max())
        
        # Top values (para categóricas)
        if profile.unique_count <= 50:  # Considerar categórica si <50 únicos
            value_counts = col_data.value_counts().head(10)
            profile.top_values = {str(k): int(v) for k, v in value_counts.items()}
        
        return profile
    
    @staticmethod
    def profile_dataframe(df: pd.DataFrame) -> Dict[str, ColumnProfile]:
        """Genera profiles de todas las columnas."""
        return {
            col: DataProfiler.profile_column(df, col) 
            for col in df.columns
        }

# 2. RULE GENERATOR

class ValidationRule(BaseModel):
    """Regla de validación generada."""
    column: str
    rule_type: str  # not_null, range, regex, in_set, unique, etc.
    parameters: Dict
    confidence: float
    reasoning: str
    great_expectations_code: str
    dbt_test_yaml: str
    pandas_assertion: str

class RuleGenerator:
    """Genera reglas de validación usando LLM."""
    
    def __init__(self, model: str = "gpt-4o"):
        self.model = model
    
    def generate_rules(
        self, 
        profile: ColumnProfile,
        business_context: Optional[str] = None
    ) -> List[ValidationRule]:
        """
        Genera reglas de validación para una columna.
        
        Args:
            profile: profile estadístico de la columna
            business_context: contexto de negocio opcional
        
        Returns:
            Lista de reglas de validación
        """
        prompt = f"""Analiza este profile de columna y genera reglas de validación apropiadas:

COLUMNA: {profile.name}
TIPO: {profile.dtype}
REGISTROS: {profile.count}
NULLS: {profile.null_count} ({profile.null_percentage:.1%})
ÚNICOS: {profile.unique_count} ({profile.unique_percentage:.1%})

{"ESTADÍSTICAS NUMÉRICAS:" if profile.mean is not None else ""}
{f"- Mean: {profile.mean:.2f}" if profile.mean is not None else ""}
{f"- Std: {profile.std:.2f}" if profile.std is not None else ""}
{f"- Range: [{profile.min}, {profile.max}]" if profile.min is not None else ""}
{f"- Q25/Q50/Q75: {profile.q25:.2f}/{profile.q50:.2f}/{profile.q75:.2f}" if profile.q25 is not None else ""}

{"ESTADÍSTICAS STRING:" if profile.avg_length is not None else ""}
{f"- Avg length: {profile.avg_length:.1f}" if profile.avg_length is not None else ""}
{f"- Max length: {profile.max_length}" if profile.max_length is not None else ""}

{"TOP VALUES:" if profile.top_values else ""}
{json.dumps(profile.top_values, indent=2) if profile.top_values else ""}

EJEMPLOS: {profile.sample_values}

{f"CONTEXTO DE NEGOCIO: {business_context}" if business_context else ""}

Genera 2-5 reglas de validación apropiadas. Para cada regla, incluye:
1. Tipo de regla (not_null, range, regex, in_set, unique, etc.)
2. Parámetros de la regla
3. Confianza (0.0-1.0) basada en qué tan claro es el patrón
4. Razonamiento (por qué esta regla tiene sentido)
5. Código de Great Expectations
6. dbt test en YAML
7. Assertion de Pandas

Responde en JSON:
{{
  "rules": [
    {{
      "rule_type": "...",
      "parameters": {{}},
      "confidence": 0.0-1.0,
      "reasoning": "...",
      "great_expectations_code": "suite.expect_...",
      "dbt_test_yaml": "- name: ...\\n  tests: ...",
      "pandas_assertion": "assert ..."
    }}
  ]
}}"""
        
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,  # Algo de creatividad pero no mucho
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            
            rules = []
            for rule_data in result.get("rules", []):
                rules.append(ValidationRule(
                    column=profile.name,
                    rule_type=rule_data["rule_type"],
                    parameters=rule_data["parameters"],
                    confidence=rule_data["confidence"],
                    reasoning=rule_data["reasoning"],
                    great_expectations_code=rule_data["great_expectations_code"],
                    dbt_test_yaml=rule_data["dbt_test_yaml"],
                    pandas_assertion=rule_data["pandas_assertion"]
                ))
            
            return rules
            
        except Exception as e:
            print(f"Error generando reglas: {e}")
            return []
    
    def generate_table_validation_suite(
        self,
        df: pd.DataFrame,
        table_name: str,
        business_context: Optional[Dict[str, str]] = None
    ) -> Dict[str, List[ValidationRule]]:
        """
        Genera suite completo de validación para tabla.
        
        Args:
            df: DataFrame a analizar
            table_name: nombre de la tabla
            business_context: diccionario {columna: contexto}
        
        Returns:
            Diccionario {columna: [reglas]}
        """
        print(f"🔍 Generando validation suite para tabla '{table_name}'...")
        
        # Profile todas las columnas
        profiler = DataProfiler()
        profiles = profiler.profile_dataframe(df)
        
        # Generar reglas para cada columna
        all_rules = {}
        business_context = business_context or {}
        
        for column, profile in profiles.items():
            print(f"  Analizando columna '{column}'...")
            
            context = business_context.get(column, "")
            rules = self.generate_rules(profile, context)
            
            all_rules[column] = rules
            print(f"    ✓ {len(rules)} reglas generadas")
        
        return all_rules

# 3. GENERADOR DE CÓDIGO EJECUTABLE

class CodeGenerator:
    """Genera código ejecutable de frameworks populares."""
    
    @staticmethod
    def generate_great_expectations_suite(
        rules_by_column: Dict[str, List[ValidationRule]],
        suite_name: str
    ) -> str:
        """Genera código de Great Expectations."""
        code = f'''"""
Auto-generated Great Expectations suite: {suite_name}
Generated by LLM-powered rule generator
"""

import great_expectations as gx
from great_expectations.core.batch import RuntimeBatchRequest

# Create expectation suite
context = gx.get_context()
suite = context.add_expectation_suite(
    expectation_suite_name="{suite_name}",
    overwrite_existing=True
)

# Expectations by column
'''
        
        for column, rules in rules_by_column.items():
            code += f"\n# {column}\n"
            for rule in rules:
                code += f"# Confidence: {rule.confidence:.0%} - {rule.reasoning}\n"
                code += f"{rule.great_expectations_code}\n"
        
        code += '''
# Save suite
context.add_or_update_expectation_suite(expectation_suite=suite)

print(f"✓ Suite '{suite_name}' created with {len(suite.expectations)} expectations")
'''
        
        return code
    
    @staticmethod
    def generate_dbt_schema_yml(
        rules_by_column: Dict[str, List[ValidationRule]],
        model_name: str
    ) -> str:
        """Genera archivo schema.yml de dbt."""
        yml = f'''version: 2

models:
  - name: {model_name}
    description: "Auto-generated dbt tests"
    columns:
'''
        
        for column, rules in rules_by_column.items():
            yml += f"      - name: {column}\n"
            yml += "        tests:\n"
            
            for rule in rules:
                # Parsear YAML del rule (simplificado)
                yml += f"          # Confidence: {rule.confidence:.0%} - {rule.reasoning}\n"
                yml += f"          {rule.dbt_test_yaml}\n"
        
        return yml
    
    @staticmethod
    def generate_pandas_validation_script(
        rules_by_column: Dict[str, List[ValidationRule]],
        script_name: str
    ) -> str:
        """Genera script de validación con Pandas."""
        code = f'''"""
Auto-generated Pandas validation script: {script_name}
"""

import pandas as pd
import sys

def validate_dataframe(df: pd.DataFrame) -> bool:
    """Valida DataFrame contra reglas generadas."""
    errors = []
    
'''
        
        for column, rules in rules_by_column.items():
            code += f"    # Validar columna '{column}'\n"
            for rule in rules:
                code += f"    # {rule.reasoning} (confidence: {rule.confidence:.0%})\n"
                code += f"    try:\n"
                code += f"        {rule.pandas_assertion}\n"
                code += f"    except AssertionError as e:\n"
                code += f"        errors.append('{column}: {{e}}')\n"
                code += f"\n"
        
        code += '''    
    if errors:
        print(f"❌ Validation failed with {len(errors)} errors:")
        for error in errors:
            print(f"  - {error}")
        return False
    
    print("✅ All validations passed!")
    return True

if __name__ == "__main__":
    # Cargar datos
    df = pd.read_csv("data.csv")
    
    # Validar
    success = validate_dataframe(df)
    
    sys.exit(0 if success else 1)
'''
        
        return code

# EJEMPLO COMPLETO

# Dataset de ejemplo
df_customers = pd.DataFrame({
    'customer_id': range(1, 101),
    'age': np.random.randint(18, 70, size=100),
    'email': [f'user{i}@example.com' for i in range(100)],
    'status': np.random.choice(['active', 'inactive', 'pending'], size=100),
    'account_balance': np.random.uniform(-100, 10000, size=100),
    'registration_date': pd.date_range('2020-01-01', periods=100)
})

# Contexto de negocio
business_context = {
    'customer_id': 'Identificador único de cliente, auto-incremental',
    'age': 'Edad del cliente, debe ser adulto (18+)',
    'email': 'Email de contacto, debe ser único y válido',
    'status': 'Estado de la cuenta del cliente',
    'account_balance': 'Balance de cuenta, puede ser negativo (deuda)',
    'registration_date': 'Fecha de registro del cliente en la plataforma'
}

# Generar reglas
generator = RuleGenerator()
rules_by_column = generator.generate_table_validation_suite(
    df_customers,
    'customers',
    business_context
)

# Mostrar reglas generadas
print("\n" + "=" * 80)
print("REGLAS DE VALIDACIÓN GENERADAS")
print("=" * 80)

for column, rules in rules_by_column.items():
    print(f"\n📊 {column}")
    for i, rule in enumerate(rules, 1):
        print(f"\n  {i}. {rule.rule_type.upper()}")
        print(f"     Confianza: {rule.confidence:.0%}")
        print(f"     Parámetros: {rule.parameters}")
        print(f"     Razonamiento: {rule.reasoning}")
        print(f"     GE: {rule.great_expectations_code[:80]}...")

# Generar código
code_gen = CodeGenerator()

print("\n" + "=" * 80)
print("CÓDIGO GENERADO")
print("=" * 80)

# Great Expectations
ge_code = code_gen.generate_great_expectations_suite(rules_by_column, 'customers_validation')
print("\n1️⃣ GREAT EXPECTATIONS:")
print(ge_code[:500] + "...\n")

# dbt
dbt_yml = code_gen.generate_dbt_schema_yml(rules_by_column, 'stg_customers')
print("\n2️⃣ DBT SCHEMA.YML:")
print(dbt_yml[:500] + "...\n")

# Pandas
pandas_script = code_gen.generate_pandas_validation_script(rules_by_column, 'validate_customers')
print("\n3️⃣ PANDAS SCRIPT:")
print(pandas_script[:500] + "...")
```

### 🔄 Refinamiento Iterativo de Reglas

```python
class RuleRefiner:
    """Refina reglas basándose en false positives/negatives."""
    
    def __init__(self, generator: RuleGenerator):
        self.generator = generator
    
    def evaluate_rule_quality(
        self,
        df: pd.DataFrame,
        rule: ValidationRule,
        ground_truth_issues: Optional[pd.Series] = None
    ) -> Dict:
        """
        Evalúa calidad de una regla ejecutándola contra datos.
        
        Args:
            df: datos a validar
            rule: regla a evaluar
            ground_truth_issues: máscara booleana de registros con issues reales
        
        Returns:
            Métricas de calidad (precision, recall, false positive rate)
        """
        # Ejecutar regla (simplificado)
        try:
            exec(rule.pandas_assertion, {"df": df})
            violations = pd.Series([False] * len(df))
        except AssertionError:
            # Regla falló, detectar qué registros violaron
            # (implementación real requiere parsear assertion)
            violations = pd.Series([True] * len(df))
        
        if ground_truth_issues is None:
            # Sin ground truth, solo reportar violations
            return {
                "violations": violations.sum(),
                "violation_rate": violations.mean(),
                "precision": None,
                "recall": None
            }
        
        # Con ground truth, calcular métricas
        true_positives = (violations & ground_truth_issues).sum()
        false_positives = (violations & ~ground_truth_issues).sum()
        false_negatives = (~violations & ground_truth_issues).sum()
        
        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        
        return {
            "violations": int(violations.sum()),
            "violation_rate": float(violations.mean()),
            "precision": precision,
            "recall": recall,
            "false_positive_rate": false_positives / len(df)
        }
    
    def refine_rule(
        self,
        rule: ValidationRule,
        quality_metrics: Dict,
        df: pd.DataFrame
    ) -> ValidationRule:
        """Refina regla si false positive rate >5%."""
        
        if quality_metrics["false_positive_rate"] > 0.05:
            print(f"⚠️  High false positive rate ({quality_metrics['false_positive_rate']:.1%}), refining rule...")
            
            # LLM sugiere ajustes
            prompt = f"""Esta regla de validación tiene demasiados false positives:

Regla: {rule.rule_type}
Parámetros: {rule.parameters}
Razonamiento: {rule.reasoning}

False positive rate: {quality_metrics['false_positive_rate']:.1%}

Analiza el profile actualizado y sugiere ajustes a los parámetros para reducir false positives.
Mantén el mismo rule_type pero ajusta thresholds, ranges, o patterns.

Responde en JSON con regla refinada."""
            
            # (implementación LLM call omitida por brevedad)
            
        return rule
```

### 📊 Comparación: Manual vs Auto-Generated Rules

| Aspecto | Manual | Auto-Generated (LLM) |
|---------|--------|---------------------|
| **Tiempo** | 2-4 horas por tabla | 5-10 minutos por tabla |
| **Cobertura** | 60-70% de columnas | 95-100% de columnas |
| **Calidad** | Alta (experto) | Media-Alta (requiere review) |
| **Mantenimiento** | Manual al cambiar datos | Re-generar automáticamente |
| **Costo** | $150-$300 (tiempo ingeniero) | $0.10-$1.00 (API calls) |
| **Consistency** | Variable por ingeniero | Consistente |

**ROI**: Auto-generación reduce tiempo **95%** y costo **99%**, pero requiere review humano.

### 💡 Mejores Prácticas

1. **Siempre revisar reglas generadas**: LLM puede inferir mal, especialmente sin contexto
2. **Proveer business context**: Mejora significativamente calidad de reglas
3. **Ejecutar en datos históricos**: Validar que reglas no tengan alta tasa de false positives
4. **Iterar**: Refinar reglas basándose en feedback de producción
5. **Versionar**: Guardar reglas en Git, no re-generar cada vez
6. **Combinar con experto**: LLM genera draft, humano refina
7. **Monitoring**: Track false positive/negative rates en producción
8. **A/B test**: Comparar LLM-generated vs manual rules en métricas de calidad

## 🔬 Root Cause Analysis: LLMs como Data Quality Investigators

Cuando las validaciones fallan, el desafío **real** no es detectar el problema sino **explicar POR QUÉ ocurrió** y **CÓMO solucionarlo**. Los LLMs actúan como **investigadores expertos** que analizan patrones, correlaciones temporales, y contexto de negocio para identificar causas raíz.

### 🕵️ Arquitectura de Root Cause Analysis

```
┌─────────────────────────────────────────────────────────────────┐
│         ROOT CAUSE ANALYSIS: FROM SYMPTOM TO SOLUTION            │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  SYMPTOM: 500 registros con email = NULL desde 2024-10-15       │
│        ↓                                                         │
│  ┌──────────────────────────────────────────────────────────┐   │
│  │ 1. TEMPORAL ANALYSIS                                     │   │
│  │    ¿Cuándo comenzó el problema?                          │   │
│  │                                                          │   │
│  │    SELECT DATE(created_at), COUNT(*) as nulls            │   │
│  │    FROM customers WHERE email IS NULL                    │   │
│  │    GROUP BY 1 ORDER BY 1 DESC                            │   │
│  │                                                          │   │
│  │    Resultado:                                            │   │
│  │    2024-10-14: 0 nulls                                   │   │
│  │    2024-10-15: 125 nulls  ← SPIKE                       │   │
│  │    2024-10-16: 187 nulls                                 │   │
│  │    2024-10-17: 188 nulls                                 │   │
│  │                                                          │   │
│  │    💡 Insight: Problema comenzó 2024-10-15               │   │
│  └──────────────────────────────────────────────────────────┘   │
│        ↓                                                         │
│  ┌──────────────────────────────────────────────────────────┐   │
│  │ 2. CORRELATION ANALYSIS                                  │   │
│  │    ¿Qué más cambió ese día?                              │   │
│  │                                                          │   │
│  │    - Git logs: Deploy de signup API v2.1 a las 14:30    │   │
│  │    - Airflow DAG 'ingest_signups' cambió lógica         │   │
│  │    - Tráfico aumentó 30% (marketing campaign)           │   │
│  │                                                          │   │
│  │    💡 Insight: Deploy coincide con inicio de nulls       │   │
│  └──────────────────────────────────────────────────────────┘   │
│        ↓                                                         │
│  ┌──────────────────────────────────────────────────────────┐   │
│  │ 3. PATTERN ANALYSIS                                      │   │
│  │    ¿Qué tienen en común los registros afectados?        │   │
│  │                                                          │   │
│  │    SELECT source, COUNT(*)                               │   │
│  │    FROM customers WHERE email IS NULL                    │   │
│  │    AND created_at >= '2024-10-15'                        │   │
│  │    GROUP BY 1                                            │   │
│  │                                                          │   │
│  │    Resultado:                                            │   │
│  │    mobile_app: 500 nulls  ← TODOS los nulls             │   │
│  │    web: 0 nulls                                          │   │
│  │                                                          │   │
│  │    💡 Insight: Solo afecta signups de mobile app         │   │
│  └──────────────────────────────────────────────────────────┘   │
│        ↓                                                         │
│  ┌──────────────────────────────────────────────────────────┐   │
│  │ 4. LLM SYNTHESIS                                         │   │
│  │    Prompt: "Analiza estos hallazgos y determina causa"  │   │
│  │                                                          │   │
│  │    LLM → ROOT CAUSE:                                     │   │
│  │    "El deploy de signup API v2.1 el 2024-10-15 introdujo│   │
│  │     un bug en el endpoint de mobile app donde el campo  │   │
│  │     'email' dejó de ser required en el request body.     │   │
│  │     El backend ahora acepta signups sin email pero la   │   │
│  │     base de datos espera NOT NULL, resultando en NULLs. │   │
│  │                                                          │   │
│  │     EVIDENCIA:                                           │   │
│  │     1. Spike exacto en fecha de deploy                   │   │
│  │     2. Solo afecta mobile app (endpoint modificado)      │   │
│  │     3. Web no afectado (endpoint diferente sin cambios)  │   │
│  │                                                          │   │
│  │     SOLUCIÓN RECOMENDADA:                                │   │
│  │     1. Rollback a API v2.0 (inmediato)                   │   │
│  │     2. Fix: Agregar validación 'email required'          │   │
│  │     3. Backfill: Contactar 500 usuarios para emails     │   │
│  │     4. Prevention: Agregar test e2e para campo required"│   │
│  └──────────────────────────────────────────────────────────┘   │
│        ↓                                                         │
│  SOLUTION: Rollback + Fix + Backfill + Prevention                │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### 🔧 Implementación Completa

```python
from typing import List, Dict, Optional
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from openai import OpenAI
from pydantic import BaseModel
import json

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# 1. MODELOS DE DATOS

class TemporalAnomaly(BaseModel):
    """Anomalía detectada con análisis temporal."""
    issue_type: str
    affected_records: int
    first_occurrence: datetime
    spike_date: Optional[datetime] = None
    trend: str  # "increasing", "decreasing", "stable", "spike"
    time_series: List[Dict[str, any]]  # [{date, count}]

class CorrelatedEvent(BaseModel):
    """Evento que coincide temporalmente con anomalía."""
    event_type: str  # "deployment", "schema_change", "traffic_spike", etc.
    timestamp: datetime
    description: str
    confidence: float  # Qué tan probable es que sea la causa

class PatternInsight(BaseModel):
    """Insight sobre patrón en datos afectados."""
    dimension: str  # Columna analizada (source, region, user_type, etc.)
    pattern: str  # Descripción del patrón
    affected_segments: Dict[str, int]  # {valor: count}
    is_significant: bool

class RootCauseHypothesis(BaseModel):
    """Hipótesis de causa raíz."""
    rank: int
    confidence: float
    root_cause: str
    evidence: List[str]
    recommended_fix: str
    prevention_steps: List[str]

class RootCauseReport(BaseModel):
    """Reporte completo de investigación."""
    issue_description: str
    temporal_analysis: TemporalAnomaly
    correlated_events: List[CorrelatedEvent]
    pattern_insights: List[PatternInsight]
    hypotheses: List[RootCauseHypothesis]
    recommended_action: str
    estimated_impact: str

# 2. ROOT CAUSE ANALYZER

class RootCauseAnalyzer:
    """Investigador automático de causas raíz."""
    
    def __init__(self, model: str = "gpt-4o"):
        self.model = model
    
    def analyze_temporal_pattern(
        self,
        df: pd.DataFrame,
        issue_column: str,
        issue_condition: str,
        date_column: str = 'created_at'
    ) -> TemporalAnomaly:
        """
        Analiza patrón temporal del issue.
        
        Args:
            df: DataFrame completo
            issue_column: columna con el issue
            issue_condition: condición para detectar issue (ej. "email IS NULL")
            date_column: columna de fecha
        """
        # Filtrar registros con issue
        issue_mask = eval(f"df['{issue_column}'].{issue_condition}")
        affected_df = df[issue_mask].copy()
        
        # Time series de issues por día
        affected_df['date'] = pd.to_datetime(affected_df[date_column]).dt.date
        daily_counts = affected_df.groupby('date').size().reset_index(name='count')
        
        # Detectar spike
        if len(daily_counts) > 1:
            mean_count = daily_counts['count'].mean()
            std_count = daily_counts['count'].std()
            spike_threshold = mean_count + (2 * std_count)
            
            spikes = daily_counts[daily_counts['count'] > spike_threshold]
            spike_date = spikes.iloc[0]['date'] if len(spikes) > 0 else None
        else:
            spike_date = None
        
        # Determinar trend
        if len(daily_counts) >= 3:
            recent_trend = daily_counts['count'].tail(3).diff().mean()
            if recent_trend > daily_counts['count'].std():
                trend = "increasing"
            elif recent_trend < -daily_counts['count'].std():
                trend = "decreasing"
            else:
                trend = "stable"
        else:
            trend = "spike" if spike_date else "stable"
        
        return TemporalAnomaly(
            issue_type=issue_condition,
            affected_records=len(affected_df),
            first_occurrence=datetime.combine(daily_counts.iloc[0]['date'], datetime.min.time()),
            spike_date=datetime.combine(spike_date, datetime.min.time()) if spike_date else None,
            trend=trend,
            time_series=[
                {"date": str(row['date']), "count": int(row['count'])}
                for _, row in daily_counts.iterrows()
            ]
        )
    
    def find_correlated_events(
        self,
        anomaly: TemporalAnomaly,
        event_log: List[Dict]
    ) -> List[CorrelatedEvent]:
        """
        Encuentra eventos que coinciden temporalmente con anomalía.
        
        Args:
            anomaly: análisis temporal
            event_log: lista de eventos [{type, timestamp, description}]
        """
        reference_date = anomaly.spike_date or anomaly.first_occurrence
        window_hours = 24  # Ventana de +/- 24 horas
        
        correlated = []
        
        for event in event_log:
            event_time = datetime.fromisoformat(event['timestamp'])
            time_diff = abs((event_time - reference_date).total_seconds() / 3600)
            
            if time_diff <= window_hours:
                # Calcular confidence basado en proximidad temporal
                confidence = 1.0 - (time_diff / window_hours)
                
                correlated.append(CorrelatedEvent(
                    event_type=event['type'],
                    timestamp=event_time,
                    description=event['description'],
                    confidence=confidence
                ))
        
        # Ordenar por confidence
        return sorted(correlated, key=lambda x: x.confidence, reverse=True)
    
    def analyze_patterns(
        self,
        df: pd.DataFrame,
        issue_column: str,
        issue_condition: str,
        dimensions: List[str]
    ) -> List[PatternInsight]:
        """
        Analiza patrones en múltiples dimensiones.
        
        Args:
            df: DataFrame completo
            issue_column: columna con issue
            issue_condition: condición de issue
            dimensions: columnas a analizar (source, region, etc.)
        """
        issue_mask = eval(f"df['{issue_column}'].{issue_condition}")
        affected_df = df[issue_mask]
        
        insights = []
        
        for dim in dimensions:
            if dim not in df.columns:
                continue
            
            # Distribución en datos afectados
            affected_dist = affected_df[dim].value_counts().to_dict()
            
            # Distribución en datos totales
            total_dist = df[dim].value_counts().to_dict()
            
            # Detectar si algún segmento está sobre-representado
            total_records = len(df)
            significant_segments = {}
            
            for value, count in affected_dist.items():
                expected_ratio = total_dist.get(value, 0) / total_records
                actual_ratio = count / len(affected_df)
                
                # Si ratio es >2x esperado, es significativo
                if actual_ratio > expected_ratio * 2:
                    significant_segments[str(value)] = count
            
            is_significant = len(significant_segments) > 0
            
            if is_significant:
                pattern = f"Segmento(s) sobre-representado(s): {', '.join(significant_segments.keys())}"
            else:
                pattern = "Distribución uniforme entre segmentos"
            
            insights.append(PatternInsight(
                dimension=dim,
                pattern=pattern,
                affected_segments={str(k): int(v) for k, v in affected_dist.items()},
                is_significant=is_significant
            ))
        
        return insights
    
    def generate_hypotheses(
        self,
        issue_description: str,
        temporal: TemporalAnomaly,
        events: List[CorrelatedEvent],
        patterns: List[PatternInsight]
    ) -> List[RootCauseHypothesis]:
        """
        Usa LLM para generar hipótesis de causa raíz.
        """
        prompt = f"""Actúa como un Data Quality Investigator experto. Analiza estos hallazgos y genera hipótesis de causa raíz:

PROBLEMA:
{issue_description}

ANÁLISIS TEMPORAL:
- Registros afectados: {temporal.affected_records}
- Primera ocurrencia: {temporal.first_occurrence}
- Spike detectado: {temporal.spike_date or "No"}
- Tendencia: {temporal.trend}
- Serie temporal: {json.dumps(temporal.time_series[:7], indent=2)}

EVENTOS CORRELACIONADOS:
{chr(10).join([f"- [{e.event_type}] {e.timestamp}: {e.description} (confidence: {e.confidence:.0%})" for e in events[:5]])}

PATRONES IDENTIFICADOS:
{chr(10).join([f"- {p.dimension}: {p.pattern}" + (f" (SIGNIFICATIVO)" if p.is_significant else "") for p in patterns])}

Genera 2-3 hipótesis de causa raíz, ordenadas por probabilidad. Para cada hipótesis:
1. Explicación de la causa raíz
2. Evidencia que la soporta
3. Fix recomendado
4. Pasos de prevención

Responde en JSON:
{{
  "hypotheses": [
    {{
      "confidence": 0.0-1.0,
      "root_cause": "explicación concisa",
      "evidence": ["evidencia 1", "evidencia 2", ...],
      "recommended_fix": "acción inmediata",
      "prevention_steps": ["paso 1", "paso 2", ...]
    }}
  ],
  "recommended_action": "qué hacer AHORA",
  "estimated_impact": "cuántos usuarios/registros/$ afectados"
}}"""
        
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            
            hypotheses = []
            for i, hyp_data in enumerate(result.get("hypotheses", []), 1):
                hypotheses.append(RootCauseHypothesis(
                    rank=i,
                    confidence=hyp_data["confidence"],
                    root_cause=hyp_data["root_cause"],
                    evidence=hyp_data["evidence"],
                    recommended_fix=hyp_data["recommended_fix"],
                    prevention_steps=hyp_data["prevention_steps"]
                ))
            
            return hypotheses, result["recommended_action"], result["estimated_impact"]
            
        except Exception as e:
            print(f"Error generando hipótesis: {e}")
            return [], "Manual investigation required", "Unknown"
    
    def investigate(
        self,
        df: pd.DataFrame,
        issue_description: str,
        issue_column: str,
        issue_condition: str,
        event_log: List[Dict],
        dimensions_to_analyze: List[str],
        date_column: str = 'created_at'
    ) -> RootCauseReport:
        """
        Investigación completa de root cause.
        
        Args:
            df: DataFrame con datos
            issue_description: descripción del problema
            issue_column: columna con issue
            issue_condition: condición que define el issue
            event_log: log de eventos (deployments, schema changes, etc.)
            dimensions_to_analyze: columnas para pattern analysis
            date_column: columna de timestamp
        
        Returns:
            Reporte completo de investigación
        """
        print("🔍 Iniciando investigación de root cause...")
        
        # 1. Análisis temporal
        print("  1️⃣ Analizando patrón temporal...")
        temporal = self.analyze_temporal_pattern(df, issue_column, issue_condition, date_column)
        print(f"     ✓ {temporal.affected_records} registros afectados, tendencia: {temporal.trend}")
        
        # 2. Eventos correlacionados
        print("  2️⃣ Buscando eventos correlacionados...")
        events = self.find_correlated_events(temporal, event_log)
        print(f"     ✓ {len(events)} eventos encontrados")
        
        # 3. Patrones
        print("  3️⃣ Analizando patrones en dimensiones...")
        patterns = self.analyze_patterns(df, issue_column, issue_condition, dimensions_to_analyze)
        significant_patterns = [p for p in patterns if p.is_significant]
        print(f"     ✓ {len(significant_patterns)}/{len(patterns)} patrones significativos")
        
        # 4. Generar hipótesis con LLM
        print("  4️⃣ Generando hipótesis de causa raíz...")
        hypotheses, recommended_action, estimated_impact = self.generate_hypotheses(
            issue_description, temporal, events, patterns
        )
        print(f"     ✓ {len(hypotheses)} hipótesis generadas")
        
        return RootCauseReport(
            issue_description=issue_description,
            temporal_analysis=temporal,
            correlated_events=events,
            pattern_insights=patterns,
            hypotheses=hypotheses,
            recommended_action=recommended_action,
            estimated_impact=estimated_impact
        )

# EJEMPLO DE USO

# Dataset simulado
np.random.seed(42)

dates = pd.date_range('2024-10-01', '2024-10-20', freq='H')
n_records = len(dates)

# Simular spike de NULLs desde 2024-10-15 solo en mobile_app
df_signups = pd.DataFrame({
    'user_id': range(n_records),
    'created_at': dates,
    'email': ['user{}@example.com'.format(i) for i in range(n_records)],
    'source': np.random.choice(['web', 'mobile_app'], size=n_records, p=[0.6, 0.4]),
    'region': np.random.choice(['US', 'EU', 'ASIA'], size=n_records)
})

# Introducir NULLs después de 2024-10-15 solo en mobile_app
spike_start = pd.Timestamp('2024-10-15')
mobile_mask = df_signups['source'] == 'mobile_app'
after_spike = df_signups['created_at'] >= spike_start

df_signups.loc[mobile_mask & after_spike, 'email'] = None

# Event log simulado
event_log = [
    {
        'type': 'deployment',
        'timestamp': '2024-10-15T14:30:00',
        'description': 'Deployed signup API v2.1 with mobile app endpoint changes'
    },
    {
        'type': 'schema_change',
        'timestamp': '2024-10-12T10:00:00',
        'description': 'Added index on users.email for performance'
    },
    {
        'type': 'traffic_spike',
        'timestamp': '2024-10-15T08:00:00',
        'description': 'Marketing campaign launch, traffic +30%'
    }
]

# Investigar
analyzer = RootCauseAnalyzer()

report = analyzer.investigate(
    df=df_signups,
    issue_description="500+ signups con email NULL desde 2024-10-15",
    issue_column='email',
    issue_condition='isnull()',
    event_log=event_log,
    dimensions_to_analyze=['source', 'region'],
    date_column='created_at'
)

# Mostrar reporte
print("\n" + "=" * 80)
print("📋 ROOT CAUSE ANALYSIS REPORT")
print("=" * 80)

print(f"\n🔴 PROBLEMA: {report.issue_description}")
print(f"\n📊 ANÁLISIS TEMPORAL:")
print(f"   Afectados: {report.temporal_analysis.affected_records}")
print(f"   Primera ocurrencia: {report.temporal_analysis.first_occurrence}")
print(f"   Spike detectado: {report.temporal_analysis.spike_date}")
print(f"   Tendencia: {report.temporal_analysis.trend}")

print(f"\n🔗 EVENTOS CORRELACIONADOS (top 3):")
for event in report.correlated_events[:3]:
    print(f"   [{event.confidence:.0%}] {event.event_type}: {event.description}")

print(f"\n🔍 PATRONES SIGNIFICATIVOS:")
for pattern in report.pattern_insights:
    if pattern.is_significant:
        print(f"   {pattern.dimension}: {pattern.pattern}")
        print(f"      Segmentos afectados: {pattern.affected_segments}")

print(f"\n💡 HIPÓTESIS DE CAUSA RAÍZ:")
for hyp in report.hypotheses:
    print(f"\n   {hyp.rank}. [{hyp.confidence:.0%}] {hyp.root_cause}")
    print(f"      Evidencia:")
    for evidence in hyp.evidence:
        print(f"        - {evidence}")
    print(f"      Fix recomendado: {hyp.recommended_fix}")
    print(f"      Prevención:")
    for step in hyp.prevention_steps:
        print(f"        - {step}")

print(f"\n🚀 ACCIÓN RECOMENDADA:")
print(f"   {report.recommended_action}")

print(f"\n📈 IMPACTO ESTIMADO:")
print(f"   {report.estimated_impact}")
```

### 📊 Métricas de Éxito en Root Cause Analysis

```python
class RCAMetrics:
    """Métricas para evaluar calidad de RCA."""
    
    @staticmethod
    def measure_time_to_root_cause(
        issue_detected_at: datetime,
        root_cause_identified_at: datetime
    ) -> float:
        """Tiempo desde detección hasta identificación de causa."""
        return (root_cause_identified_at - issue_detected_at).total_seconds() / 3600  # horas
    
    @staticmethod
    def measure_hypothesis_accuracy(
        predicted_cause: str,
        actual_cause: str,
        llm_model: str = "gpt-4o"
    ) -> float:
        """
        Evalúa similitud semántica entre causa predicha y real.
        Retorna score 0.0-1.0.
        """
        # Usar embeddings para similitud semántica
        from openai import OpenAI
        client = OpenAI()
        
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=[predicted_cause, actual_cause]
        )
        
        emb1 = np.array(response.data[0].embedding)
        emb2 = np.array(response.data[1].embedding)
        
        # Cosine similarity
        similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
        
        return float(similarity)

# Ejemplo de medición
metrics = RCAMetrics()

# Manual investigation: 4 horas
manual_time = 4.0

# LLM-powered: 15 minutos
llm_time = 0.25

print(f"⚡ Speedup: {manual_time / llm_time:.0f}x más rápido")
print(f"💰 Ahorro: ${manual_time * 75:.0f} (@ $75/hora ingeniero) vs ${0.50:.2f} (API calls)")
```

### 💡 Mejores Prácticas

1. **Siempre investigar**: No asumir causa, seguir proceso sistemático
2. **Múltiples hipótesis**: LLM debe generar 2-3 opciones, no solo una
3. **Evidencia cuantitativa**: Basar hipótesis en datos, no suposiciones
4. **Timeline crítico**: Marcar eventos importantes (deploys, schema changes)
5. **Documentar**: Guardar investigaciones para aprendizaje futuro
6. **Validar hipótesis**: Confirmar causa raíz antes de aplicar fix masivo
7. **Post-mortem automático**: Generar documento de incident con LLM
8. **Learn from patterns**: Entrenar modelo en investigaciones previas

## 🏭 LLM-Powered Data Quality en Producción

Implementar LLMs para Data Quality en producción requiere arquitectura robusta, monitoring exhaustivo, y estrategias para controlar costos y latencia. Aquí exploramos patrones de producción reales.

### 🏗️ Arquitectura de Data Quality Platform con LLMs

```
┌─────────────────────────────────────────────────────────────────┐
│         DATA QUALITY PLATFORM: HYBRID ARCHITECTURE               │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌──────────────────────────────────────────────────────────┐   │
│  │ INGESTION LAYER                                          │   │
│  │  • Kafka / Kinesis streams                               │   │
│  │  • Airflow DAGs                                          │   │
│  │  • dbt models                                            │   │
│  └────────────────┬─────────────────────────────────────────┘   │
│                   ↓                                              │
│  ┌──────────────────────────────────────────────────────────┐   │
│  │ VALIDATION LAYER (Multi-Stage)                           │   │
│  │                                                          │   │
│  │  Stage 1: TRADITIONAL (100% traffic, <10ms)             │   │
│  │  ┌────────────────────────────────────────────────┐     │   │
│  │  │ • NULL checks (Pandas)                         │     │   │
│  │  │ • Range validation (SQL)                       │     │   │
│  │  │ • Regex patterns (Python re)                   │     │   │
│  │  │ • Referential integrity (JOIN queries)         │     │   │
│  │  │                                                │     │   │
│  │  │ ✅ Pass → Continue                             │     │   │
│  │  │ ❌ Fail → Block + Alert                        │     │   │
│  │  └────────────────────────────────────────────────┘     │   │
│  │                   ↓                                      │   │
│  │  Stage 2: LLM SEMANTIC (1% sample, ~500ms)              │   │
│  │  ┌────────────────────────────────────────────────┐     │   │
│  │  │ • Semantic NULL detection                      │     │   │
│  │  │ • Plausibility checks                          │     │   │
│  │  │ • PII detection                                │     │   │
│  │  │ • Anomaly explanation                          │     │   │
│  │  │                                                │     │   │
│  │  │ Cache Layer (Redis):                           │     │   │
│  │  │  key: hash(value + context)                    │     │   │
│  │  │  value: {is_valid, confidence, reasoning}      │     │   │
│  │  │  TTL: 7 days                                   │     │   │
│  │  │                                                │     │   │
│  │  │ ⚠️  Issues → Queue for investigation           │     │   │
│  │  └────────────────────────────────────────────────┘     │   │
│  │                   ↓                                      │   │
│  │  Stage 3: ROOT CAUSE ANALYSIS (on failures)             │   │
│  │  ┌────────────────────────────────────────────────┐     │   │
│  │  │ • Triggered solo si >threshold failures        │     │   │
│  │  │ • Analiza temporal patterns                    │     │   │
│  │  │ • Correlaciona con eventos (deploys, etc.)     │     │   │
│  │  │ • Genera hipótesis con LLM                     │     │   │
│  │  │ • Crea ticket automático (Jira)               │     │   │
│  │  └────────────────────────────────────────────────┘     │   │
│  └──────────────────────────────────────────────────────────┘   │
│                   ↓                                              │
│  ┌──────────────────────────────────────────────────────────┐   │
│  │ MONITORING & ALERTING                                    │   │
│  │  • Prometheus metrics                                    │   │
│  │  • Grafana dashboards                                    │   │
│  │  • PagerDuty/Slack alerts                                │   │
│  │  • Cost tracking (LLM API usage)                         │   │
│  └──────────────────────────────────────────────────────────┘   │
│                   ↓                                              │
│  ┌──────────────────────────────────────────────────────────┐   │
│  │ DATA STORAGE                                             │   │
│  │  • Validated data → Data Warehouse (Snowflake)           │   │
│  │  • Validation logs → S3 → Athena queries                 │   │
│  │  • Failed records → Dead Letter Queue (DLQ)              │   │
│  └──────────────────────────────────────────────────────────┘   │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### 🔧 Implementación: Production-Grade System

```python
from typing import Dict, List, Optional
import asyncio
import redis
import hashlib
import json
from datetime import datetime, timedelta
from dataclasses import dataclass
import logging
from prometheus_client import Counter, Histogram, Gauge
from openai import AsyncOpenAI

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Prometheus metrics
VALIDATIONS_TOTAL = Counter('dq_validations_total', 'Total validations', ['stage', 'result'])
VALIDATION_DURATION = Histogram('dq_validation_duration_seconds', 'Validation duration', ['stage'])
LLM_API_COST = Gauge('dq_llm_api_cost_usd', 'LLM API cost')
CACHE_HIT_RATE = Gauge('dq_cache_hit_rate', 'Cache hit rate')

# 1. CACHE LAYER

class ValidationCache:
    """Redis-backed cache para validaciones LLM."""
    
    def __init__(self, redis_url: str = "redis://localhost:6379"):
        self.redis_client = redis.from_url(redis_url, decode_responses=True)
        self.ttl_seconds = 7 * 24 * 3600  # 7 días
        self.hits = 0
        self.misses = 0
    
    def _get_cache_key(self, value: str, context: str, validation_type: str) -> str:
        """Genera cache key determinístico."""
        content = f"{validation_type}:{context}:{value}"
        return f"dq:validation:{hashlib.sha256(content.encode()).hexdigest()}"
    
    def get(self, value: str, context: str, validation_type: str) -> Optional[Dict]:
        """Obtiene resultado cacheado."""
        key = self._get_cache_key(value, context, validation_type)
        
        cached = self.redis_client.get(key)
        
        if cached:
            self.hits += 1
            CACHE_HIT_RATE.set(self.hits / (self.hits + self.misses))
            return json.loads(cached)
        else:
            self.misses += 1
            CACHE_HIT_RATE.set(self.hits / (self.hits + self.misses))
            return None
    
    def set(self, value: str, context: str, validation_type: str, result: Dict):
        """Cachea resultado."""
        key = self._get_cache_key(value, context, validation_type)
        self.redis_client.setex(key, self.ttl_seconds, json.dumps(result))

# 2. VALIDATION ORCHESTRATOR

@dataclass
class ValidationConfig:
    """Configuración de validación."""
    traditional_enabled: bool = True
    llm_enabled: bool = True
    llm_sample_rate: float = 0.01  # 1% de tráfico
    llm_max_concurrent: int = 10
    llm_timeout_seconds: int = 5
    cost_budget_daily_usd: float = 10.0
    failure_threshold_for_rca: int = 100  # Trigger RCA si >100 failures

class ProductionValidator:
    """Validador de producción con stages."""
    
    def __init__(
        self,
        config: ValidationConfig,
        cache: ValidationCache,
        llm_client: AsyncOpenAI
    ):
        self.config = config
        self.cache = cache
        self.llm_client = llm_client
        self.daily_cost = 0.0
        self.last_cost_reset = datetime.now()
    
    def _should_use_llm(self) -> bool:
        """Decide si usar LLM basado en sampling y budget."""
        # Reset daily cost
        if datetime.now() - self.last_cost_reset > timedelta(days=1):
            self.daily_cost = 0.0
            self.last_cost_reset = datetime.now()
        
        # Check budget
        if self.daily_cost >= self.config.cost_budget_daily_usd:
            logger.warning(f"Daily LLM budget exhausted: ${self.daily_cost:.2f}")
            return False
        
        # Sample rate
        import random
        return random.random() < self.config.llm_sample_rate
    
    async def validate_record(
        self,
        record: Dict,
        validation_rules: Dict[str, Dict]
    ) -> Dict:
        """
        Valida un registro completo.
        
        Args:
            record: diccionario con datos del registro
            validation_rules: reglas por columna
                {
                    "column_name": {
                        "traditional": {...},
                        "llm": {"enabled": bool, "context": str}
                    }
                }
        
        Returns:
            {
                "valid": bool,
                "issues": List[Dict],
                "stages_executed": List[str],
                "duration_ms": float,
                "cost_usd": float
            }
        """
        start_time = datetime.now()
        issues = []
        stages_executed = []
        total_cost = 0.0
        
        # STAGE 1: TRADITIONAL (ALWAYS)
        if self.config.traditional_enabled:
            with VALIDATION_DURATION.labels(stage='traditional').time():
                trad_issues = await self._validate_traditional(record, validation_rules)
                issues.extend(trad_issues)
                stages_executed.append('traditional')
                VALIDATIONS_TOTAL.labels(stage='traditional', result='pass' if len(trad_issues)==0 else 'fail').inc()
        
        # Si traditional falla críticamente, no continuar
        critical_issues = [i for i in issues if i.get('severity') == 'CRITICAL']
        if critical_issues:
            duration_ms = (datetime.now() - start_time).total_seconds() * 1000
            return {
                "valid": False,
                "issues": issues,
                "stages_executed": stages_executed,
                "duration_ms": duration_ms,
                "cost_usd": 0.0
            }
        
        # STAGE 2: LLM SEMANTIC (SAMPLED)
        if self.config.llm_enabled and self._should_use_llm():
            with VALIDATION_DURATION.labels(stage='llm_semantic').time():
                llm_issues, llm_cost = await self._validate_llm_semantic(record, validation_rules)
                issues.extend(llm_issues)
                stages_executed.append('llm_semantic')
                total_cost += llm_cost
                self.daily_cost += llm_cost
                LLM_API_COST.set(self.daily_cost)
                VALIDATIONS_TOTAL.labels(stage='llm_semantic', result='pass' if len(llm_issues)==0 else 'fail').inc()
        
        duration_ms = (datetime.now() - start_time).total_seconds() * 1000
        
        return {
            "valid": len(issues) == 0,
            "issues": issues,
            "stages_executed": stages_executed,
            "duration_ms": duration_ms,
            "cost_usd": total_cost
        }
    
    async def _validate_traditional(
        self,
        record: Dict,
        rules: Dict
    ) -> List[Dict]:
        """Stage 1: validación tradicional (rápida)."""
        issues = []
        
        for column, rule_config in rules.items():
            if 'traditional' not in rule_config:
                continue
            
            value = record.get(column)
            trad_rules = rule_config['traditional']
            
            # NULL check
            if trad_rules.get('not_null') and value is None:
                issues.append({
                    "column": column,
                    "issue_type": "NULL",
                    "severity": "CRITICAL",
                    "message": f"Column '{column}' cannot be NULL"
                })
            
            # Range check
            if 'range' in trad_rules and value is not None:
                min_val, max_val = trad_rules['range']
                if not (min_val <= value <= max_val):
                    issues.append({
                        "column": column,
                        "issue_type": "RANGE",
                        "severity": "ERROR",
                        "message": f"Value {value} outside range [{min_val}, {max_val}]"
                    })
            
            # Regex check
            if 'regex' in trad_rules and value is not None:
                import re
                if not re.match(trad_rules['regex'], str(value)):
                    issues.append({
                        "column": column,
                        "issue_type": "FORMAT",
                        "severity": "ERROR",
                        "message": f"Value '{value}' doesn't match expected format"
                    })
        
        return issues
    
    async def _validate_llm_semantic(
        self,
        record: Dict,
        rules: Dict
    ) -> tuple[List[Dict], float]:
        """Stage 2: validación semántica con LLM (muestreada)."""
        issues = []
        total_cost = 0.0
        
        # Validar columnas con LLM enabled
        llm_columns = {
            col: config for col, config in rules.items()
            if config.get('llm', {}).get('enabled', False)
        }
        
        # Crear tasks concurrentes (con límite)
        semaphore = asyncio.Semaphore(self.config.llm_max_concurrent)
        
        async def validate_column(column: str, config: Dict):
            async with semaphore:
                value = record.get(column)
                if value is None:
                    return None, 0.0
                
                context = config['llm']['context']
                
                # Check cache
                cached = self.cache.get(str(value), context, 'plausibility')
                if cached:
                    if not cached['is_valid']:
                        return {
                            "column": column,
                            "issue_type": "IMPLAUSIBLE",
                            "severity": "WARNING",
                            "message": f"{cached['reasoning']} (cached)",
                            "confidence": cached['confidence']
                        }, 0.0
                    return None, 0.0
                
                # LLM validation
                prompt = f"""Valida plausibilidad:
Column: {column}
Context: {context}
Value: {value}

¿Es válido? Responde JSON: {{"is_valid": bool, "confidence": 0-1, "reasoning": "..."}}"""
                
                try:
                    response = await asyncio.wait_for(
                        self.llm_client.chat.completions.create(
                            model="gpt-4o-mini",
                            messages=[{"role": "user", "content": prompt}],
                            temperature=0,
                            response_format={"type": "json_object"}
                        ),
                        timeout=self.config.llm_timeout_seconds
                    )
                    
                    result = json.loads(response.choices[0].message.content)
                    
                    # Estimar costo
                    cost = (len(prompt) / 4 / 1_000_000 * 0.15) + (len(response.choices[0].message.content) / 4 / 1_000_000 * 0.60)
                    
                    # Cache result
                    self.cache.set(str(value), context, 'plausibility', result)
                    
                    if not result['is_valid']:
                        return {
                            "column": column,
                            "issue_type": "IMPLAUSIBLE",
                            "severity": "WARNING",
                            "message": result['reasoning'],
                            "confidence": result['confidence']
                        }, cost
                    
                    return None, cost
                    
                except asyncio.TimeoutError:
                    logger.warning(f"LLM timeout for column {column}")
                    return None, 0.0
                except Exception as e:
                    logger.error(f"LLM error for column {column}: {e}")
                    return None, 0.0
        
        # Ejecutar validaciones en paralelo
        tasks = [validate_column(col, config) for col, config in llm_columns.items()]
        results = await asyncio.gather(*tasks)
        
        for issue, cost in results:
            if issue:
                issues.append(issue)
            total_cost += cost
        
        return issues, total_cost

# 3. BATCH VALIDATOR (para procesamiento masivo)

class BatchValidator:
    """Validador optimizado para batches grandes."""
    
    def __init__(self, validator: ProductionValidator):
        self.validator = validator
    
    async def validate_batch(
        self,
        records: List[Dict],
        validation_rules: Dict,
        parallelism: int = 100
    ) -> Dict:
        """
        Valida batch de registros en paralelo.
        
        Returns:
            {
                "total_records": int,
                "valid_records": int,
                "invalid_records": int,
                "issues_by_type": Dict[str, int],
                "total_duration_seconds": float,
                "total_cost_usd": float
            }
        """
        start_time = datetime.now()
        
        # Semaphore para controlar paralelismo
        semaphore = asyncio.Semaphore(parallelism)
        
        async def validate_with_semaphore(record):
            async with semaphore:
                return await self.validator.validate_record(record, validation_rules)
        
        # Validar todos los registros
        tasks = [validate_with_semaphore(record) for record in records]
        results = await asyncio.gather(*tasks)
        
        # Agregar resultados
        valid_count = sum(1 for r in results if r['valid'])
        invalid_count = len(results) - valid_count
        
        issues_by_type = {}
        total_cost = 0.0
        
        for result in results:
            total_cost += result['cost_usd']
            for issue in result['issues']:
                issue_type = issue['issue_type']
                issues_by_type[issue_type] = issues_by_type.get(issue_type, 0) + 1
        
        duration = (datetime.now() - start_time).total_seconds()
        
        return {
            "total_records": len(records),
            "valid_records": valid_count,
            "invalid_records": invalid_count,
            "issues_by_type": issues_by_type,
            "total_duration_seconds": duration,
            "total_cost_usd": total_cost,
            "throughput_records_per_second": len(records) / duration
        }

# EJEMPLO DE USO EN PRODUCCIÓN

async def main():
    # Setup
    cache = ValidationCache()
    llm_client = AsyncOpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    
    config = ValidationConfig(
        traditional_enabled=True,
        llm_enabled=True,
        llm_sample_rate=0.01,  # 1% de tráfico
        llm_max_concurrent=10,
        cost_budget_daily_usd=10.0
    )
    
    validator = ProductionValidator(config, cache, llm_client)
    batch_validator = BatchValidator(validator)
    
    # Reglas de validación
    validation_rules = {
        'email': {
            'traditional': {
                'not_null': True,
                'regex': r'^[\w\.-]+@[\w\.-]+\.\w+$'
            },
            'llm': {
                'enabled': True,
                'context': 'Customer emails (no test data)'
            }
        },
        'age': {
            'traditional': {
                'not_null': True,
                'range': [18, 120]
            }
        }
    }
    
    # Dataset de prueba (1000 registros)
    records = [
        {'email': f'user{i}@example.com', 'age': 25 + (i % 50)}
        for i in range(1000)
    ]
    
    # Agregar algunos registros problemáticos
    records[10]['email'] = 'test@test.test'  # Fake email
    records[20]['age'] = 999  # Outlier
    records[30]['email'] = None  # NULL
    
    # Validar batch
    print("🚀 Validando batch de 1000 registros...")
    result = await batch_validator.validate_batch(records, validation_rules, parallelism=50)
    
    print("\n📊 RESULTADOS:")
    print(f"   Total: {result['total_records']}")
    print(f"   Válidos: {result['valid_records']} ({result['valid_records']/result['total_records']:.1%})")
    print(f"   Inválidos: {result['invalid_records']}")
    print(f"   Issues por tipo: {result['issues_by_type']}")
    print(f"   Duración: {result['total_duration_seconds']:.2f}s")
    print(f"   Throughput: {result['throughput_records_per_second']:.0f} records/s")
    print(f"   Costo total: ${result['total_cost_usd']:.4f}")

# Ejecutar
# asyncio.run(main())
```

### 📊 Métricas de Producción

| Métrica | Target | Medición |
|---------|--------|----------|
| **Latency p50** | <50ms | Traditional only |
| **Latency p99** | <500ms | With LLM (1% sample) |
| **Throughput** | >1000 records/s | Batch processing |
| **Cache hit rate** | >80% | Redis cache |
| **Cost per 1M records** | <$10 | LLM API calls |
| **False positive rate** | <2% | Validation precision |
| **Availability** | 99.9% | Uptime |

### 💰 Estrategias de Optimización de Costos

```python
# 1. ADAPTIVE SAMPLING
class AdaptiveSampler:
    """Ajusta sample rate basándose en calidad de datos."""
    
    def __init__(self, initial_rate: float = 0.01):
        self.rate = initial_rate
        self.recent_issues = []
    
    def update(self, has_issues: bool):
        """Actualiza rate basándose en issues recientes."""
        self.recent_issues.append(has_issues)
        
        # Mantener ventana de últimos 1000 registros
        if len(self.recent_issues) > 1000:
            self.recent_issues = self.recent_issues[-1000:]
        
        issue_rate = sum(self.recent_issues) / len(self.recent_issues)
        
        # Si issue rate es alta (>5%), aumentar sampling
        if issue_rate > 0.05:
            self.rate = min(0.10, self.rate * 1.5)
        # Si issue rate es baja (<1%), reducir sampling
        elif issue_rate < 0.01:
            self.rate = max(0.001, self.rate * 0.8)
    
    def should_sample(self) -> bool:
        import random
        return random.random() < self.rate

# 2. SMART CACHING
# Cachear por más tiempo valores que se repiten frecuentemente
# TTL adaptativo basado en frecuencia

# 3. BATCH INFERENCE
# Procesar múltiples validaciones en un solo prompt
# Reducir overhead de API calls
```

### 🚀 Deployment Checklist

- [ ] **Infraestructura**:
  - [ ] Redis cluster para cache (multi-AZ)
  - [ ] Async workers (Celery / RQ)
  - [ ] Load balancer para distribución
  
- [ ] **Monitoring**:
  - [ ] Prometheus + Grafana dashboards
  - [ ] Alertas en PagerDuty/Slack
  - [ ] Cost tracking diario
  - [ ] Latency monitoring (p50, p95, p99)
  
- [ ] **Testing**:
  - [ ] Load testing (>10K records/s)
  - [ ] Chaos testing (LLM API down)
  - [ ] Cost simulation
  
- [ ] **Security**:
  - [ ] API key rotation
  - [ ] PII masking en logs
  - [ ] Rate limiting por usuario
  
- [ ] **Documentation**:
  - [ ] Runbook para incidents
  - [ ] SLA commitments
  - [ ] Cost budgets por equipo

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 1. Detección de anomalías semánticas

In [ ]:
import os
import pandas as pd
from openai import OpenAI

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def detect_anomaly(value: str, context: str) -> dict:
    """Detecta si un valor es anómalo en su contexto."""
    prompt = f'''
Contexto: {context}
Valor: {value}

¿Es este valor anómalo o incorrecto? Responde en JSON:
{{
  "is_anomaly": true/false,
  "confidence": 0-100,
  "reason": "explicación",
  "suggested_fix": "valor corregido o null"
}}
'''
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0
    )
    import json
    return json.loads(resp.choices[0].message.content)

# Ejemplos
casos = [
    {'valor': 'Nueva Yorkk', 'contexto': 'Columna: ciudad (ciudades de USA)'},
    {'valor': '999', 'contexto': 'Columna: edad (años de personas)'},
    {'valor': 'admin@example.com', 'contexto': 'Columna: email de clientes reales'}
]

for caso in casos:
    result = detect_anomaly(caso['valor'], caso['contexto'])
    print(f"Valor: {caso['valor']}")
    print(f"Anomalía: {result['is_anomaly']} (confianza={result['confidence']}%)")
    print(f"Razón: {result['reason']}")
    if result['suggested_fix']:
        print(f"Sugerencia: {result['suggested_fix']}")
    print()

## 2. Clasificación de errores en datos

In [ ]:
def classify_data_issue(issue_description: str) -> str:
    """Clasifica el tipo de problema de datos."""
    prompt = f'''
Clasifica este problema de datos en UNA categoría:
- DUPLICATES: registros duplicados
- NULLS: valores faltantes
- FORMAT: formato incorrecto
- OUTLIER: valores fuera de rango
- INCONSISTENCY: datos inconsistentes entre fuentes
- FRESHNESS: datos desactualizados

Problema: {issue_description}

Categoría:
'''
    resp = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{'role':'user','content':prompt}],
        temperature=0
    )
    return resp.choices[0].message.content.strip()

issues = [
    'La tabla tiene 500 filas con cliente_id = NULL',
    'Fechas en formato DD/MM/YYYY pero esperamos YYYY-MM-DD',
    'Última actualización hace 7 días pero debe ser diaria',
    'Misma transacción aparece 3 veces con diferentes IDs'
]

for issue in issues:
    categoria = classify_data_issue(issue)
    print(f'➡️ "{issue}"')
    print(f'   Categoría: {categoria}\n')

## 3. Generación de reglas de validación

In [ ]:
def generate_validation_rules(column_name: str, sample_data: list, description: str = '') -> str:
    """Genera reglas de Great Expectations."""
    prompt = f'''
Genera expectativas de Great Expectations (Python) para validar esta columna:

Columna: {column_name}
Descripción: {description}
Muestra de datos: {sample_data}

Genera código Python con expect_* methods. Ejemplos:
- expect_column_values_to_not_be_null
- expect_column_values_to_be_between
- expect_column_values_to_match_regex

Código:
'''
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.1
    )
    return resp.choices[0].message.content.strip().replace('```python','').replace('```','')

# Ejemplo
rules = generate_validation_rules(
    column_name='email',
    sample_data=['user@example.com', 'admin@test.org', 'info@company.co'],
    description='Emails de clientes'
)

print(rules)

## 4. Validación batch con LLM

In [ ]:
def validate_batch(df: pd.DataFrame, rules: dict) -> pd.DataFrame:
    """Valida DataFrame según reglas inferidas por LLM."""
    results = []
    
    for col, rule_desc in rules.items():
        sample = df[col].dropna().head(10).tolist()
        
        prompt = f'''
Valida si estos valores cumplen la regla:
Regla: {rule_desc}
Valores: {sample}

Responde con porcentaje de conformidad (0-100) y problemas encontrados.
'''
        
        resp = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[{'role':'user','content':prompt}],
            temperature=0
        )
        
        results.append({
            'columna': col,
            'regla': rule_desc,
            'resultado': resp.choices[0].message.content
        })
    
    return pd.DataFrame(results)

# Datos de prueba
df_test = pd.DataFrame({
    'edad': [25, 30, 200, 45, -5],
    'email': ['a@b.com', 'invalido', 'test@x.org', None, 'ok@mail.com']
})

validation_rules = {
    'edad': 'Debe estar entre 0 y 120',
    'email': 'Debe ser email válido o null'
}

validation_report = validate_batch(df_test, validation_rules)
print(validation_report)

## 5. Explicación de anomalías

In [ ]:
def explain_outlier(value: float, stats: dict) -> str:
    """Explica por qué un valor es outlier en lenguaje natural."""
    prompt = f'''
Valor: {value}
Estadísticas de la columna:
- Media: {stats['mean']}
- Desviación estándar: {stats['std']}
- Min: {stats['min']}
- Max: {stats['max']}

Explica en 2-3 frases por qué este valor es anómalo y qué puede indicar.
'''
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.3
    )
    return resp.choices[0].message.content.strip()

# Ejemplo
outlier_explanation = explain_outlier(
    value=50000,
    stats={'mean': 120, 'std': 35, 'min': 50, 'max': 250}
)

print('Explicación del outlier:')
print(outlier_explanation)

## 6. Sugerencia de limpieza de datos

In [ ]:
def suggest_data_cleaning(df_sample: pd.DataFrame) -> str:
    """Sugiere pasos de limpieza basados en muestra."""
    info = {
        'columns': df_sample.columns.tolist(),
        'dtypes': df_sample.dtypes.astype(str).to_dict(),
        'nulls': df_sample.isnull().sum().to_dict(),
        'sample': df_sample.head(3).to_dict()
    }
    
    prompt = f'''
Analiza este DataFrame y sugiere pasos de limpieza en orden de prioridad:

{info}

Lista numerada de acciones de limpieza con código Pandas cuando sea relevante.
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.2
    )
    
    return resp.choices[0].message.content

messy_df = pd.DataFrame({
    'fecha': ['2024-01-01', '01/02/2024', None, '2024-03-15'],
    'monto': ['100', '200.5', 'N/A', '300'],
    'categoria': ['  ventas', 'VENTAS', 'Ventas ', 'marketing']
})

cleaning_plan = suggest_data_cleaning(messy_df)
print(cleaning_plan)

## 7. Validación de coherencia entre tablas

In [ ]:
def validate_referential_integrity(parent_ids: list, child_ids: list, relationship: str) -> dict:
    """Valida integridad referencial con explicación."""
    orphans = set(child_ids) - set(parent_ids)
    
    prompt = f'''
Relación: {relationship}
IDs huérfanos (en tabla hija pero no en padre): {list(orphans)[:10]}
Total huérfanos: {len(orphans)}

Explica el problema y sugiere 3 posibles causas.
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.2
    )
    
    return {
        'orphans_count': len(orphans),
        'orphan_sample': list(orphans)[:5],
        'explanation': resp.choices[0].message.content
    }

# Ejemplo
productos_ids = [1, 2, 3, 4, 5]
ventas_producto_ids = [1, 2, 3, 99, 100, 5]

integrity_check = validate_referential_integrity(
    parent_ids=productos_ids,
    child_ids=ventas_producto_ids,
    relationship='ventas.producto_id -> productos.producto_id'
)

print(f"Huérfanos: {integrity_check['orphans_count']}")
print(f"Muestra: {integrity_check['orphan_sample']}")
print(f"\nExplicación:\n{integrity_check['explanation']}")

## 8. Buenas prácticas

- **Complementar, no reemplazar**: LLMs complementan herramientas tradicionales (GE, pandas profiling).
- **Validación humana**: revisa sugerencias antes de aplicar.
- **Umbrales**: define confidence thresholds para automatización.
- **Logging**: registra todas las decisiones del LLM.
- **Costos**: cachea validaciones comunes.
- **Testing**: valida el validador con datos conocidos.

## 9. Ejercicios

1. Construye un sistema de auto-reparación de datos usando LLM suggestions.
2. Genera un data quality dashboard con explicaciones en lenguaje natural.
3. Implementa detección de PII (datos sensibles) con LLMs.
4. Crea un agente que diagnostique problemas de data quality y proponga fixes.

---

## 🧭 Navegación

**← Anterior:** [🤖 Agentes Autónomos para Automatización](06_agentes_automatizacion.ipynb)

**Siguiente →:** [🎲 Generación de Datos Sintéticos con LLMs →](08_sintesis_aumento_datos.ipynb)

**📚 Índice de Nivel GenAI:**
- [🔄 Comparación: OpenAI vs Google Gemini](00_comparacion_openai_gemini.ipynb)
- [🤖 Fundamentos de LLMs y Prompting](01_fundamentos_llms_prompting.ipynb)
- [📊 Text-to-SQL: Generación de Consultas SQL desde Lenguaje Natural](02_generacion_sql_nl2sql.ipynb)
- [🔧 Generación Automática de Código ETL con LLMs](03_generacion_codigo_etl.ipynb)
- [📚 RAG: Documentación Técnica con LLMs](04_rag_documentacion_datos.ipynb)
- [🔍 Embeddings y Similitud en Datos](05_embeddings_similitud_datos.ipynb)
- [🤖 Agentes Autónomos para Automatización](06_agentes_automatizacion.ipynb)
- [✅ Validación de Datos con LLMs](07_calidad_validacion_llm.ipynb) ← 🔵 Estás aquí
- [🎲 Generación de Datos Sintéticos con LLMs](08_sintesis_aumento_datos.ipynb)
- [🚀 Proyecto Integrador 1: Chatbot de Consulta de Datos con RAG](09_proyecto_integrador_1.ipynb)
- [🏗️ Proyecto Integrador 2: Plataforma Self-Service con GenAI](10_proyecto_integrador_2.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
